# Q learning for connect four
In this document the use of the reinforment leanring _Q learning_ will be tested. an algrithm for connect four will be trained. as an opponent the connect four AI will be used.

## Algrithmn
Q-learning is a reinforcement learning technique used in machine learning. The technique does not require a model of the environment. Q-learning can handle problems with stochastic transitions and rewards, without requiring adaptations.[1]
Q learning builds a Q matrix of states, transitions (actions) and the evaluation of these.

In this example the scores the node (status) and the game actions (inserting a game chip in the player's color into a column) the transitions. If a player has won in a score, the final node is reached. Formally this can be disengaged like next (with a learnrate $\alpha$ and a discovery rate $\gamma$):

$
Q(s_{t},a_{t}) = (1-\alpha) \cdot Q(s_{t},a_{t}) + \alpha \bigg( \gamma \underbrace{\max_{i} Q(s_{t+1}, a_{i})}_{\rm estimate~of~optimal~future~value} \bigg)
$


The algrithm now adjusts the transitions during teaching, so that the maximum path to a final state is reached. During learning the conditions are visited and examined randomly.

The final algrithm selects the action with the highest transition value from the Q matrix.

### Adaptions to connect four 
For the problem examined here, some adjustments have to be made. Instead, the Q matrix is updated and played against another AI over and over again. However, due to the large number of stands, not a small part of the nodes are visited. Therefore, at the end of a game, all nodes in the Q matrix are updated.

It is also problematic to allocate a matrix for all possible game states. Therefore, a dictionary was used here. To model the state of the game the Horner's method[2] is used. If $b_{i}$ is the value of the playing field $i$ with red equals 1, white equals 2 and empty equals 0, this can be formulated as follows:

$\sum_{i}^{42} b_{i} 3^{i}$

### Requirements
For this example the ConnectFourAILib is used. You can download it [here](https://github.com/jeroen-andress/ConnectFourAILib). It must be compiled and installed for Python3.

## Summary
* The algrithm always selects only the action with the highest rating. therefore, however, we do not consider the advantage of against. if, for example, the opponent has won in a turn, a turn is still chosen that leads to a victory.

* Due to the very large search space (small compared to chess or go) many training games are necessary. 

* The arithmetic is also able to teach during test phase.

## Author
Jeroen Andreß, M.Sc. Professional software developer.

## References
* [1]: https://en.wikipedia.org/wiki/Q-learning
* [2]: https://en.wikipedia.org/wiki/Horner%27s_method
* [Reinforcement Learning - A Simple Python Example and a Step Closer to AI with Assisted Q-Learning](http://amunategui.github.io/reinforcement-learning)
* [A Painless Q-Learning Tutorial](http://mnemstudio.org/path-finding-q-learning-tutorial.htm)


In [1]:
from ConnectFourAI import ConnectFourPlayboard
from ConnectFourAI import ConnectFourPlayerMinimaxAlphaBeta
from ConnectFourAI import ConnectFourPlayerMinimax
from ConnectFourAI import ConnectFourPlayerInterface

import numpy as np

from IPython import display

### Helper functions
The following method is used to display the board.

In [2]:
from ipythonblocks import BlockGrid, colors, fui_colors

def show_playboard(playboard):
    grid = BlockGrid(playboard.GetColumns(), playboard.GetRows(), fill=fui_colors.MidnightBlue)
    
    for r in range(playboard.GetRows()):
        for c in range(playboard.GetColumns()):
            if playboard.GetField(r, c) == playboard.WHITE:
                grid[r, c] = fui_colors.Clouds
            if playboard.GetField(r, c) == playboard.RED:
                grid[r, c] = fui_colors.Pomegranate

    grid.show()

Functions to detect actions on playboard and to en/decode playboard from/to a number represtion.

In [3]:
def available_actions(playboard):
    return [i for i in range(playboard.GetColumns()) if playboard.IsInsertAble(i)]

def encode_playboard(playboard):
    result = 0
    for i in range(playboard.GetRows()):
        for j in range(playboard.GetColumns()):
            r = playboard.GetRows() - i - 1
            c = j 

            result += playboard.GetField(r, c) * 3 ** (i * playboard.GetColumns() + c)

    return result

def decode_playboard(v):
    # Only for debugging
    result = ConnectFourPlayboard()

    for i in range(result.GetRows()):
        for c in range(result.GetColumns()):
            if (v % 3 == ConnectFourPlayboard.RED):
                result.MoveRed(c)
            elif (v % 3 == ConnectFourPlayboard.WHITE):
                result.MoveWhite(c)
            v = int(v / 3)

    return result

In [4]:
class ConnectFourQLearningPlayer(ConnectFourPlayerInterface):
    def __init__(self, player):
        # public attribute
        self.playboard = None
        
        # protected
        self._player = player
        self._Q = {}

    # Impl interface
    def GetPlayer(self):
        return self._player
    
    def MakeMove(self):
        assert self.playboard is not None
        action, v = self._next_action(self.playboard)
        self.playboard.Move(int(action), self._player)
       
        return action
     
    # Internal functions
    def _sample_next_action(self, playboard):
        return int(np.random.choice(available_actions(playboard)))
    
    def _next_action(self, playboard):
        encoded = encode_playboard(playboard)
        
        if encoded in self._Q:
            actions = self._Q[encoded]
            return (np.argmax(actions), np.max(actions))
        else:
            actions = available_actions(playboard)
            return (np.random.choice(actions) if len(actions) > 0 else 0, -1)
    
    def _copy_and_insert_into_playboard(self, playboard, column, player):
        result = ConnectFourPlayboard(playboard)
        result.Move(column, player)
    
        return result
    
    def _get_against_players(self, player):
        return ConnectFourPlayboard.RED if player == ConnectFourPlayboard.WHITE else ConnectFourPlayboard.RED

    def _R(self, playboard, player):
        if playboard.GetWinner() == player:
            return 1.0
        elif playboard.GetWinner() == ConnectFourPlayboard.NONE:
            return 0.0
        else:
            return -1.0

    def _get_following_states(self, playboard, player):
        for i in available_actions(playboard):
            yield (i, self._copy_and_insert_into_playboard(column=i, playboard=playboard, player=player))    

    def _update(self, Q, gamma, learning_rate, state, action, againt_action, player):
        state_exe = self._copy_and_insert_into_playboard(state, action, player)
        state_encoded = encode_playboard(state)

        if state_encoded not in Q:
            Q[state_encoded] = np.full(state_exe.GetColumns(), 0.0)

        if state.GetWinner() == ConnectFourPlayboard.NONE:
            maxQ = None

            for i in self._get_following_states(state_exe, self._get_against_players(player)):
                encode_following_state = encode_playboard(i[1])

                if encode_following_state in Q:
                    if maxQ is None:
                        maxQ = Q[encode_following_state][againt_action]
                    else:
                        maxQ = max(maxQ, Q[encode_following_state][againt_action])
            
            Q[state_encoded][action] = (1.0 - learning_rate) * Q[state_encoded][action] + learning_rate * self._R(state_exe, player) + gamma * maxQ if maxQ is not None else 0.0
        else:
            Q[state_encoded][action] = (1.0 - learning_rate) * Q[state_encoded][action] + learning_rate * self._R(state_exe, player)
            
    def fit(self, againt_player_creator, gamma = 0.8, learning_rate = 0.01, iterations = 10 ** 5, useQ = False):
        winners = {ConnectFourPlayboard.WHITE: 0, ConnectFourPlayboard.RED: 0, ConnectFourPlayboard.NONE: 0}
        
        create_action = self._sample_next_action if not useQ else lambda playboard: self._next_action(playboard)[0]
        
        for i in range(iterations):
            playboard = ConnectFourPlayboard()
            against_player = againt_player_creator(playboard, self._get_against_players(self._player))
            assert against_player.GetPlayer() == self._get_against_players(self._player)
            
            history = []

            while playboard.GetWinner() == ConnectFourPlayboard.NONE and not playboard.IsFull():
                against_player.MakeMove()
                action = create_action(playboard)
                history.append((ConnectFourPlayboard(playboard), int(action)))
                playboard.Move(int(action), self._player)

            for j in range(len(history) - 1, -1, -1):
                #print('Update:', history[j][1], encode_playboard(history[j][0]), history[j+1][1] if (j + 1) < len(history) else -111111 )
                #show_playboard(history[j][0])
                self._update(Q=self._Q, gamma=gamma, learning_rate=learning_rate, action=history[j][1], againt_action=history[j+1][1] if (j + 1) < len(history) else None, player=self._player, state=history[j][0])


            display.clear_output(wait=True)
            winners[playboard.GetWinner()] += 1
            relativeWinners = lambda p: 100.0 * float(winners[p]) / float(np.sum(list(winners.values())))
            print('(%3d%%): Winner is Red: %5d (%3d%%), Winner is White: %5d (%3d%%), No winner: %5d (%3d%%)' % (float(i + 1) / float(iterations) * 100.0, winners[ConnectFourPlayboard.RED], relativeWinners(ConnectFourPlayboard.RED), winners[ConnectFourPlayboard.WHITE], relativeWinners(ConnectFourPlayboard.WHITE), winners[ConnectFourPlayboard.NONE], relativeWinners(ConnectFourPlayboard.NONE)))
                       

            show_playboard(playboard)

        for q in sorted(self._Q):
            print("%20d:\t%s" % (q, ','.join(['%5.4f' %i for i in self._Q[q]])))

## Training
1. Train with a random player. Use to Q matrix to get better results
2. Train with KI (Minimax algrithmn) without Q matrix (random actions) to finalize the Q matrix

In [5]:
class ConnectFourRandomPlayer(ConnectFourPlayerInterface):
    def __init__(self, playboard, player):
        # protected
        self._playboard = playboard
        self._player = player
        self._Q = {}

    # Impl interface
    def GetPlayer(self):
        return self._player
    
    def MakeMove(self):
        column = int(np.random.choice(available_actions(self._playboard)))
        self._playboard.Move(column, self._player)
        return column
        

In [6]:
player = ConnectFourQLearningPlayer(player=ConnectFourPlayboard.WHITE)

iterations = 10 ** 3
gamma = 0.8
learning_rate = 0.1

In [7]:
trainerRandom = lambda playboard, player: ConnectFourRandomPlayer(playboard, player)
player.fit(iterations=iterations, againt_player_creator=trainerRandom, useQ=True)

(100%): Winner is Red:   521 ( 52%), Winner is White:   478 ( 47%), No winner:     1 (  0%)


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


                   1:	0.4595,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   3:	6.2720,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   9:	12.7251,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  14:	0.2796,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  27:	-0.0009,-0.0021,-0.0033,1.0685,0.0000,0.0000,0.0000
                  32:	0.1490,0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000
                  38:	0.9438,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  81:	10.3547,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  86:	0.2350,-0.0033,0.0000,0.0000,0.0000,0.0000,0.0000
                  92:	-0.0011,0.0020,0.0000,0.0000,0.0000,0.0000,0.0000
                 110:	0.4466,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                 125:	-0.0064,0.0000,-0.0009,0.0000,0.0000,0.0000,0.0000
                 243:	14.4350,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                 248:	0.1702,0.0000,0.0000,0.0000,0.0000,-0.0009,0

              545687:	0.0000,0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000
              545695:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
              549914:	-0.0041,-0.0017,0.0000,-0.0009,0.0000,0.0000,0.0000
              550007:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
              553628:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
              555764:	-0.0013,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
              556541:	0.0000,0.0000,0.0000,0.0011,0.0000,0.0000,0.0000
              562373:	0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000
              591911:	0.0000,0.0000,0.0000,-0.0026,0.0000,0.0000,0.0000
              595889:	0.0000,0.0000,0.0000,-0.0051,0.0000,0.0000,0.0000
              596195:	0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000,0.0000
              609017:	0.0000,-0.0011,0.0000,0.0000,0.0000,0.0000,0.0000
              649811:	-0.0051,0.0000,-0.0051,0.0000,0.0000,0.0000,0.0000
              649898:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000

             4795124:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0026
             4795493:	0.0000,-0.0021,0.0000,0.0000,0.0000,0.0000,0.0000
             4799498:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             4799543:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             4800722:	-0.0064,-0.0009,0.0000,0.0000,0.0000,0.0000,0.0000
             4801228:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
             4801280:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             4801325:	0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000,0.0000
             4801397:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             4801535:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             4801553:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
             4806322:	0.0000,0.0000,-0.0009,0.0000,0.0000,0.0000,0.0000
             4820162:	-0.0064,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0033
             4820377:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0

...

   17127304744059140:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17165945532790570:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17173074231401074:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17302792317629834:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0033,0.0000
   17307876049286501:	0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000
   17307876049306190:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
   17928066255930719:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   17930896272783412:	0.0000,0.0000,0.0000,0.0000,-0.0051,0.0000,0.0000
   18547162390368656:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   20557249720187819:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0064
   27815654039573216:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
   27838530832028195:	0.0000,0.0000,0.0000,-0.0051,0.0000,0.0000,0.0000
   29705954158107710:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
   29706048387379979:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.000

   50090024554825019:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
   50092563838796918:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50092563838837034:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
   50092565002121267:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50094353392167038:	0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000
   50097648776824646:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50097648791186675:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50123125516463315:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50123125516502708:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50123126678882273:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50138391000243896:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50146002394869391:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50146003560319504:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50146021692712636:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
  

  150209632717340045:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150224358850989794:	0.0000,0.0000,0.0017,0.0000,0.0000,0.0000,0.0000
  150225731200919462:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231927464686907:	0.0000,0.0000,0.0000,-0.0017,0.0000,0.0000,0.0000
  150231927464824688:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0021,0.0000
  150231927470670539:	0.0000,0.0000,0.0000,0.0000,-0.0026,0.0000,0.0000
  150231930957455102:	0.0000,0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000
  150231932032933903:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231934425151250:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231934429934237:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231935204794898:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231955362151298:	-0.0051,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231958829798822:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  150231958834586165:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 

 4054409889175540327:	0.0000,0.0000,-0.0026,0.0000,0.0000,0.0000,0.0000
 4054410464686285253:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0100
 4054425125603634332:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
 4054432860128153329:	0.0000,-0.0013,0.0000,0.0000,0.0000,0.0000,0.0000
 4054615968593992630:	0.0000,0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000
 4054638751260261100:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0017
 4054638758362970065:	0.0000,-0.0021,0.0000,0.0000,0.0000,0.0000,0.0000
 4054638789772727488:	0.0000,-0.0026,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262069490874250:	0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262069505236279:	-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262888371250154:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262888401542291:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262919782641266:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 4056262930285084013:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0

13119614457729209602:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
13121469172495280329:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
13571012105121187436:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
13620479360907107698:	0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000,0.0000
13920668631501125623:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
13920668725730397892:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
14865551380969575818:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
14865619992386175284:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
14921781608558511970:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
15015760381268084327:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
16625409884712822866:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
17174024325517445242:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
17174024332620154207:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
17174024928860286778:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
203

In [8]:
trainerMinimax = lambda playboard, player: ConnectFourPlayerMinimaxAlphaBeta(playboard, player, 4)
player.fit(iterations=iterations, gamma=gamma, learning_rate=learning_rate, againt_player_creator=trainerMinimax)

(100%): Winner is Red:   995 ( 99%), Winner is White:     5 (  0%), No winner:     0 (  0%)


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


                   1:	0.4595,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   3:	6.2720,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                   9:	12.7251,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  14:	0.2796,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  27:	2.5043,0.0088,-0.0056,0.1241,-5.3506,-8.6353,-17.0643
                  32:	0.1490,0.0000,0.0000,0.0000,-0.0013,0.0000,0.0000
                  38:	0.9438,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  81:	10.3547,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                  86:	0.2350,-0.0033,0.0000,0.0000,0.0000,0.0000,0.0000
                  92:	-0.0011,0.0020,0.0000,0.0000,0.0000,0.0000,0.0000
                 110:	0.4466,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                 125:	-0.0064,0.0000,-0.0009,0.0000,0.0000,0.0000,0.0000
                 243:	14.4350,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                 248:	0.1702,0.0000,0.0000,0.0000,0.0000,-0.0009

                7783:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                8295:	0.0000,0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000
                8394:	0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000
                8837:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0017
                9239:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
                9485:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               10949:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               10967:	0.0123,-0.0026,0.0000,0.0000,0.0000,0.0000,0.0000
               10969:	0.0017,-0.0011,0.0000,0.0000,0.0000,0.0000,0.0000
               11021:	0.0127,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               11066:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0017
               11068:	0.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
               11183:	-0.0005,0.0063,0.0000,0.0000,0.0000,0.0000,0.0000
               11669:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000

             1634516:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1636901:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
             1641724:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1645397:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0033,0.0000
             1654154:	0.0000,0.0000,-0.0021,0.0000,0.0000,0.0000,0.0000
             1659089:	-0.0005,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1695707:	-0.0026,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1713183:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1713266:	0.0000,-0.0009,0.0000,0.0000,0.0000,0.0000,0.0000
             1713428:	0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000
             1715777:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1717819:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000
             1717913:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
             1719780:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

...

   50398026227945990:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50398874291396411:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50466352991935252:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50468108655571595:	0.0000,0.0000,-0.0051,0.0000,0.0000,0.0000,0.0000
   50489283232664051:	0.0000,0.0000,0.0000,-0.0033,0.0000,0.0000,0.0000
   50489283232664186:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
   50489290208514572:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
   50489314646673515:	0.0000,0.0000,0.0000,0.0000,-0.0041,0.0000,0.0000
   50489315808935144:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051,0.0000
   50494399540768949:	0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000
   50494399928307536:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
   50535387324407933:	0.0000,0.0000,0.0000,0.0080,0.0000,0.0000,0.0000
   50695590264875842:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
   50696960232781240:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000

  100109042598273488:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
  100109046501281194:	0.0000,0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000
  100109053038025282:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0033
  100109053428634417:	0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000,0.0000
  100109054330614265:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
  100109137587210340:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
  100109147396456942:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  100109147398090631:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  100109154371665994:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  100109231816482609:	0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000
  100109701974942590:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000
  100109705530110011:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0100,0.0000
  100109708933105572:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
  100109733841153592:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0

 2702322238992690437:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0800
 2702322584183291282:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
 2702323086667667143:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 2702323379375925823:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.1000,0.0000
 2702323999194903088:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 2702324012358106232:	0.0000,0.0000,0.0000,0.0000,-0.0410,0.0000,0.0000
 2702324860421556653:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0512
 2702336743668790481:	0.0000,0.0000,-0.0064,0.0000,0.0000,0.0000,0.0000
 2702336930108056526:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
 2702344571455812262:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
 2702344578816801553:	0.0000,0.0000,0.0000,0.0000,-0.0064,0.0000,0.0000
 2702346274556281906:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
 2702348816422149601:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0100
 2702374889127870949:	0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000,0.

28380863752277712380:	0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000,0.0000
28381618686428726093:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
28482800742662067860:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0064
28482800742679605413:	0.0000,0.0080,0.0000,0.0000,0.0000,0.0000,0.0000
28482800805484771352:	0.0000,0.0000,0.0000,0.0100,0.0000,0.0000,0.0000
28677435401454007768:	-0.0041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
28777498585795186009:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0051
28777513837119296146:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0064
28811074091650723933:	0.0000,0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000
28811692329906080842:	0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000
28881431297293771310:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
30078129278651527754:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
33347199823465126426:	0.0000,-0.0100,0.0000,0.0000,0.0000,0.0000,0.0000
36489686276754396230:	0.0000,0.0000,-0.0051,0.0000,0.0000,0.0000,0.00

44665375959935325251:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
44865451117064834149:	0.0000,0.0000,0.0000,0.0000,0.0000,-0.0080,0.0000
44876569332341124022:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
44898969949806544370:	-0.0064,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
44899015703520594455:	0.0000,0.0000,-0.0080,0.0000,0.0000,0.0000,0.0000
44948818414690026371:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
44948818415852287892:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
44948823499584003599:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
44949029579002455902:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
45410431606026564311:	0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
45849615060401588888:	0.0000,0.0000,0.0000,-0.0100,0.0000,0.0000,0.0000
48652908883033466167:	0.0000,0.0000,-0.0033,0.0000,0.0000,0.0000,0.0000
48652908977262738436:	0.0000,0.0000,0.0000,-0.0041,0.0000,0.0000,0.0000
48654350215187400979:	0.0000,0.0000,0.0000,-0.0051,0.0000,0.0000,0.00

## Test

In [9]:
playboard = ConnectFourPlayboard()

player.playboard = playboard

players = [ConnectFourPlayerMinimax(playboard, ConnectFourPlayboard.RED, 1), player]

while playboard.GetWinner() == ConnectFourPlayboard.NONE and not playboard.IsFull():
    move = [0, 0]
    for i in range(len(players)):
        move[i] = players[i].MakeMove()
    
    print('Player0 = %d, Player1 = %d' % (move[0], move[1]))
    show_playboard(playboard)
    
print('The Winner is: %s' % 'Red' if playboard.GetWinner() == ConnectFourPlayboard.RED else ('White' if playboard.GetWinner() == ConnectFourPlayboard.WHITE else 'nobody'))
    

Player0 = 3, Player1 = 0


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 6


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 1


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


Player0 = 3, Player1 = 1


,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,
,,,,,,


The Winner is: Red
